In [8]:
import pymysql
import requests

### 下載分析pm2.5 opendata 資料

In [11]:
url="https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON"

In [12]:
resp=requests.get(url,verify=False)
resp

c:\Users\USER\Desktop\shuan\MySQL\pm25_tomysql\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [13]:
resp.json()

{'fields': [{'id': 'site', 'type': 'text', 'info': {'label': '測站名稱'}},
  {'id': 'county', 'type': 'text', 'info': {'label': '縣市名稱'}},
  {'id': 'pm25', 'type': 'text', 'info': {'label': '細懸浮微粒濃度'}},
  {'id': 'datacreationdate', 'type': 'text', 'info': {'label': '資料建置日期'}},
  {'id': 'itemunit', 'type': 'text', 'info': {'label': '測項單位'}}],
 'resource_id': 'c1f31192-babd-4105-b880-a4c2e23a3276',
 '__extras': {'api_key': '540e2ca4-41e1-4186-8497-fdd67024ac44'},
 'include_total': True,
 'total': '1289626',
 'resource_format': 'object',
 'limit': '1000',
 'offset': '0',
 '_links': {'start': '/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&format=JSON&sort=datacreationdate desc',
  'next': '/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&format=JSON&sort=datacreationdate desc&offset=1000'},
 'records': [{'site': '員林',
   'county': '彰化縣',
   'pm25': '4',
   'datacreationdate': '2025-08-20 19:00',
   'itemunit': 'μg/m3'},
  {'site': '大城',
   'cou

In [15]:
datas=resp.json()['records']
datas

[{'site': '員林',
  'county': '彰化縣',
  'pm25': '4',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '大城',
  'county': '彰化縣',
  'pm25': '5',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '富貴角',
  'county': '新北市',
  'pm25': '5',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '麥寮',
  'county': '雲林縣',
  'pm25': '1',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '關山',
  'county': '臺東縣',
  'pm25': '4',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '馬公',
  'county': '澎湖縣',
  'pm25': '2',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '金門',
  'county': '金門縣',
  'pm25': '',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '馬祖',
  'county': '連江縣',
  'pm25': '13',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '埔里',
  'county': '南投縣',
  'pm25': '2',
  'datacreationdate':

In [16]:
len(datas)

1000

In [17]:
datas[0]

{'site': '員林',
 'county': '彰化縣',
 'pm25': '4',
 'datacreationdate': '2025-08-20 19:00',
 'itemunit': 'μg/m3'}

### 建立資料表

In [18]:
table_str="""
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20)
)
"""
print(table_str)



create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20)
)



### 連接資料庫

In [20]:
conn=pymysql.connect(
    host="localhost",
    # host="127.0.0.1",
    user="root",
    password="",
    port=3307,
    database="demo"
)

cursor=conn.cursor()
cursor

In [21]:
cursor.execute(table_str)
conn.commit()

### 寫入一筆資料

In [25]:
sqlstr="insert into pm25 (site,county,pm25,datacreationdate,itemunit)\
      values('{}','{}',{},'{}','{}')"

data=list(datas[0].values())
print(data)
sqlstr.format(data[0],data[1],data[2],data[3],data[4])


['員林', '彰化縣', '4', '2025-08-20 19:00', 'μg/m3']


"insert into pm25 (site,county,pm25,datacreationdate,itemunit)      values('員林','彰化縣',4,'2025-08-20 19:00','μg/m3')"

In [27]:
cursor.execute(sqlstr.format(data[0],data[1],data[2],data[3],data[4]))
conn.commit()

### 一次寫入

In [ ]:
sqlstr="insert into pm25 (site,county,pm25,datacreationdate,itemunit)\
      values(%s,%s,%s,%s,%s)"

values=[list(data.values()) for data in datas if list(data.values())[2]!=""]
# values=[list(data.values()) for data in datas]

cursor.executemany(sqlstr,values)
conn.commit()